# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [71]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [72]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [73]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [74]:
# check column headers in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print([line for line in f][0])

"artist","firstName","gender","itemInSession","lastName","length","level","location","sessionId","song","userId"





## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [75]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

import cassandra

from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [76]:
# Creating a keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [77]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Initiate a list of table names 

In [78]:
tables = ['session_item','user_item','user_song']

### Drop tables if they exists in the list

In [79]:
for table in tables:
    query = "drop TABLE IF EXISTS "
    query += table 
    print(query)
    try:
        session.execute(query)
    except Exception as e:
        print(e)

drop TABLE IF EXISTS session_item
drop TABLE IF EXISTS user_item
drop TABLE IF EXISTS user_song


### Create tables from a list of tables

In [80]:
# count is a variable to check index of table name in the initiated tables list
count = 0

for table in tables:
    query = "CREATE TABLE IF NOT EXISTS "
    query += table
    if count == 0:
        
# For Query 1: Table must be partitioned by columns (sessionId, itemInSession) because the query filters results on columns (sessionId, itemInSession)
        query += """ (sessionId int, itemInSession int, artist text, firstName text, gender text,
lastName text, length float, level text, location text, song text, userId int,  PRIMARY KEY (sessionId, itemInSession))"""
    
# For Query 2: must be partitioned by (userId, sessionId) and clustered for sorting by itemInSession
    elif count == 1:
        query += """ (userId int, sessionId int, itemInSession int, artist text, firstName text, gender text, 
lastName text, length float, level text, location text, song text, PRIMARY KEY ((userId, sessionId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC)"""
        
# For Query 3: must be partitioned by song and userId
    elif count == 2:
        query += """ (song text, userId int, artist text, firstName text, gender text, itemInSession int, sessionId int, lastName text, length float, level text, location text, PRIMARY KEY (song, userId))"""
    count +=1
    print(query)

    try:
        session.execute(query)
    except Exception as e:
        print(e)

CREATE TABLE IF NOT EXISTS session_item (sessionId int, itemInSession int, artist text, firstName text, gender text,
lastName text, length float, level text, location text, song text, userId int,  PRIMARY KEY (sessionId, itemInSession))
CREATE TABLE IF NOT EXISTS user_item (userId int, sessionId int, itemInSession int, artist text, firstName text, gender text, 
lastName text, length float, level text, location text, song text, PRIMARY KEY ((userId, sessionId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC)
CREATE TABLE IF NOT EXISTS user_song (song text, userId int, artist text, firstName text, gender text, itemInSession int, sessionId int, lastName text, length float, level text, location text, PRIMARY KEY (song, userId))


### Populate created above tables with data from provided file

In [81]:
count = 0

file = 'event_datafile_new.csv'

for table in tables:
    query = "INSERT INTO "+table

    if count == 0:

        query += " (sessionId, itemInSession, artist , firstName , gender , lastName , length , level , location ,  song , userId) "
        query += " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        
        with open(file, encoding = 'utf8') as f:
            csvreader = csv.reader(f)
            next(csvreader) # skip header
            for line in csvreader:
                try:
                    session.execute(query, (int(line[8]),int(line[3]),line[0],line[1],line[2],line[4],float(line[5]),line[6],line[7],line[9],int(line[10])))
                except Exception as e:
                    print(e)

    elif count == 1:

        query += " (userId, sessionId, itemInSession, artist, firstName , gender , lastName , length , level , location ,  song ) "
        query += " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        
        with open(file, encoding = 'utf8') as f:
            csvreader = csv.reader(f)
            next(csvreader) # skip header
            for line in csvreader:
                try:
                    session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[1],line[2],line[4],float(line[5]),line[6],line[7],line[9]))
                except Exception as e:
                    print(e)

    elif count == 2:

        query += " (song, userId, artist , firstName , gender, itemInSession, sessionId, lastName , length , level , location) "
        query += " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        
        with open(file, encoding = 'utf8') as f:
            csvreader = csv.reader(f)
            next(csvreader) # skip header
            for line in csvreader:
                try:
                    session.execute(query, (line[9],int(line[10]),line[0],line[1],line[2],int(line[3]),int(line[8]),line[4],float(line[5]),line[6],line[7]))
                except Exception as e:
                    print(e)

    count +=1            
            

### Check number of rows inserted into each table

In [82]:
for table in tables:
    query = "select count(*) from " +table
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

    for row in rows:
        print (row)

Row(count=6820)
Row(count=6820)
Row(count=6618)


### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [83]:
query = "select artist, song, length from session_item where sessionId=338 and  itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [84]:
query = "select artist, song, firstName, lastName from user_item where userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)
                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [85]:
query = "select firstname, lastname from user_song where song ='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ( row.firstname, row.lastname)

                    
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [86]:
for table in tables:
    query = "drop TABLE IF EXISTS "
    query += table 
    print(query)
    try:
        session.execute(query)
    except Exception as e:
        print(e)

drop TABLE IF EXISTS session_item
drop TABLE IF EXISTS user_item
drop TABLE IF EXISTS user_song


### Close the session and cluster connection¶

In [48]:
session.shutdown()
cluster.shutdown()